# GAGE II Modeling - Data Preparation

-------------------
### Author: Yunsu Park
### Date: 2025-01-23
---------------------

This notebook prepares the data for the GAGE II modeling. 

To reproduce the result of this notebook, make sure the the following folders are in the same directory as this notebook:

- `GL_daymet_1980-2023`: contains the climate variables for the GAGE II modeling 
- `discharge/Canada`: a __folder__ containing the discharge data for the Canadian stations
- `discharge/GL_USA_USGS_streamflow.csv`: a __file__ containing the discharge data for the US stations

All of the data listed above are available in the project Googld Drive.


# Canadian Basin Data Preparation

## Data Cleaning for discharge variable

This following code cell processes multiple CSV files containing discharge data, cleans the data, and exports the cleaned data to a specified directory.

In [7]:
import os
import pandas as pd
from tqdm import tqdm

def process_discharge_data(input_folder: str, output_folder: str):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Create a complete date range from '1980/01/01' to '2023/12/31'
    date_range = pd.date_range(start='1980-01-01', end='2023-12-31')
    date_range_df = pd.DataFrame(date_range, columns=['Date'])
    
    # Iterate through all CSV files in the input folder
    for file_name in tqdm(os.listdir(input_folder)):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_folder, file_name)
            df = pd.read_csv(file_path, skiprows=1)
            
            # Filter rows where PARAM is 1
            df_filtered = df[df['PARAM'] == 1]
            
            # Group by ID and save each group to a separate CSV file
            for gage_id, group in df_filtered.groupby(' ID'):
                # Convert the 'Date' column to datetime
                group['Date'] = pd.to_datetime(group['Date'], format='%Y/%m/%d')
                
                # Merge with the complete date range
                merged_df = pd.merge(date_range_df, group[['Date', 'Value']], on='Date', how='left')
                merged_df.rename(columns={'Value': 'discharge'}, inplace=True)
                
                # Export the merged data
                output_file_path = os.path.join(output_folder, f"{gage_id.strip()}.csv")
                merged_df.to_csv(output_file_path, index=False)

# Example usage
input_folder = 'discharge/Canada'
output_folder = 'Canadian_basins_discharge_cleaned'
process_discharge_data(input_folder, output_folder)

  0%|          | 0/41 [00:00<?, ?it/s]

100%|██████████| 41/41 [00:22<00:00,  1.80it/s]


## Combine climate data with discharge data

Now that the discharge data is sorted out, we can combine the discharge data with the climate data. 

The following code cell reads the discharge data and the climate data, and combines them into a single dataframe. The combined dataframe is then exported to a specified directory.

Also, since the climate data is availalbe from 1980 to 2023, we will only use the discharge data from 1980 to 2023.

In [2]:
from glob import glob

# Define the directories
discharge_dir = r"Canadian_basins_discharge_cleaned"
climate_dir = r"GL_daymet_1980-2023"
output_dir = r"Canadian_climate_discharge_data"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get all discharge files and extract basin IDs
discharge_files = glob(os.path.join(discharge_dir, ".csv"))
basin_ids = [os.path.basename(f).split('_')[1].split('.')[0] for f in discharge_files]

print(basin_ids)

[]


In [9]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm

# Define the directories
discharge_dir = r"Canadian_basins_discharge_cleaned"
climate_dir = r"GL_daymet_1980-2023"
output_dir = r"Canadian_climate_discharge_data"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get all discharge files and extract basin IDs
discharge_files = glob(os.path.join(discharge_dir, "*.csv"))
basin_ids = [os.path.basename(f).split('.')[0] for f in discharge_files]

# Loop through each basin ID
for basin_id in tqdm(basin_ids, desc="Processing basins"):
    # Read the discharge data
    discharge_file = os.path.join(discharge_dir, f"{basin_id}.csv")
    discharge_data = pd.read_csv(discharge_file)
    discharge_data['Date'] = pd.to_datetime(discharge_data['Date'], format='%Y-%m-%d')
    discharge_data.rename(columns={'Value': 'discharge'}, inplace=True)
    
    # Initialize a list to hold climate data
    climate_data_list = []
    
    # Loop through each subfolder (variable) in the climate directory
    for subfolder in os.listdir(climate_dir):
        subfolder_path = os.path.join(climate_dir, subfolder)
        if os.path.isdir(subfolder_path):
            # Check if the climate file exists for the current basin ID
            climate_file = os.path.join(subfolder_path, f"{basin_id}_daymet_{subfolder.split('_')[-1]}.csv")
            if os.path.exists(climate_file):
                # Read the climate data
                climate_data = pd.read_csv(climate_file)
                climate_data['time'] = pd.to_datetime(climate_data['time']).dt.strftime('%Y-%m-%d')
                climate_data.rename(columns={'time': 'Date', subfolder.split('_')[-1]: subfolder.split('_')[-1]}, inplace=True)
                climate_data['Date'] = pd.to_datetime(climate_data['Date'], format='%Y-%m-%d')
                climate_data_list.append(climate_data)
    
    # Merge all climate data with discharge data on 'Date'
    combined_data = discharge_data
    for climate_data in climate_data_list:
        combined_data = pd.merge(combined_data, climate_data, on='Date', how='inner')
    
    # Drop rows with dates earlier than 1980/01/01
    combined_data = combined_data[combined_data['Date'] >= '1980/01/01']
    
    # Export the combined data
    output_file = os.path.join(output_dir, f"{basin_id}.csv")
    combined_data.to_csv(output_file, index=False)

Processing basins: 100%|██████████| 577/577 [04:50<00:00,  1.99it/s]


### Check if the processed data has all the necessary columns

In [6]:
# Specify the necessary columns
necessary_columns = ['Date', 'discharge', 'dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp'] 

# Path to the folder containing the CSV files
folder_path = 'Canadian_climate_discharge_data'

# List to store the results
results = []

# Loop through all files in the folder
for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        missing_columns = [col for col in necessary_columns if col not in df.columns]
        if missing_columns:
            results.append(f"{file_name} is missing columns: {', '.join(missing_columns)}")
        

# Print the result message
print("Looping through CSV files is done.")

if len(results) == 0:
    print("All files contain necessary columns.")
else:
    for result in results:
        print(result)

100%|██████████| 40/40 [00:03<00:00, 10.65it/s]

Looping through CSV files is done.
All files contain necessary columns.


# US Basins

## Combine climate data with discharge data

The US discharge data is available in a single CSV file. 

Therefore, we can directly combine the discharge data with the climate data, skipping the data sorting process as we did for the Canadian basins.

The following code cell reads the discharge data and the climate data, and combines them into a single dataframe per basin. The combined dataframes are then exported to a specified directory.

In [3]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm

# Define the directories
discharge_file = r"discharge\GL_USA_USGS_streamflow.csv"
climate_dir = r"GL_daymet_1980-2023"
output_dir = r"US_climate_discharge_data"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the discharge data
discharge_data = pd.read_csv(discharge_file)
discharge_data['datetime'] = pd.to_datetime(discharge_data['datetime'])
discharge_data['Date'] = discharge_data['datetime'].dt.strftime('%Y/%m/%d')
discharge_data.drop(columns=['datetime'], inplace=True)

# Get all basin IDs from the discharge data columns
basin_ids = discharge_data.columns.drop('Date')

# Loop through each basin ID
for basin_id in tqdm(basin_ids, desc="Processing basins"):
    # Extract the discharge data for the current basin ID
    basin_discharge_data = discharge_data[['Date', basin_id]].rename(columns={basin_id: 'discharge'})
    basin_discharge_data['Date'] = pd.to_datetime(basin_discharge_data['Date'], format='%Y/%m/%d')
    
    # Initialize a list to hold climate data
    climate_data_list = []
    
    # Loop through each subfolder (variable) in the climate directory
    for subfolder in os.listdir(climate_dir):
        subfolder_path = os.path.join(climate_dir, subfolder)
        if os.path.isdir(subfolder_path):
            # Check if the climate file exists for the current basin ID
            climate_file = os.path.join(subfolder_path, f"{basin_id}_daymet_{subfolder.split('_')[-1]}.csv")
            if os.path.exists(climate_file):
                # Read the climate data
                climate_data = pd.read_csv(climate_file)
                climate_data['time'] = pd.to_datetime(climate_data['time']).dt.strftime('%Y/%m/%d')
                climate_data.rename(columns={'time': 'Date', subfolder.split('_')[-1]: subfolder.split('_')[-1]}, inplace=True)
                climate_data['Date'] = pd.to_datetime(climate_data['Date'], format='%Y/%m/%d')
                climate_data_list.append(climate_data)
    
    # Merge all climate data with discharge data on 'Date'
    combined_data = basin_discharge_data
    for climate_data in climate_data_list:
        combined_data = pd.merge(combined_data, climate_data, on='Date', how='inner')
    
    # Drop rows with dates earlier than 1980/01/01
    combined_data = combined_data[combined_data['Date'] >= '1980/01/01']
    
    # Export the combined data
    output_file = os.path.join(output_dir, f"{basin_id}.csv")
    combined_data.to_csv(output_file, index=False)

Processing basins: 100%|██████████| 399/399 [04:46<00:00,  1.39it/s]


### Check if the processed data has all the necessary columns

In [8]:
# Specify the necessary columns
necessary_columns = ['Date', 'discharge', 'dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp'] 

# Path to the folder containing the CSV files
folder_path = 'US_climate_discharge_data'

# List to store the results
results = []

# Loop through all files in the folder
for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        missing_columns = [col for col in necessary_columns if col not in df.columns]
        if missing_columns:
            results.append(f"{file_name} is missing columns: {', '.join(missing_columns)}")
        

# Print the result message
print("Looping through CSV files is done.")

if len(results) == 0:
    print("All files contain necessary columns.")
else:
    for result in results:
        print(result)

100%|██████████| 399/399 [00:07<00:00, 55.71it/s]

Looping through CSV files is done.
All files contain necessary columns.
